In [138]:
""" Prepare a datasetof 5 objects with all kind of attributes seen
    Nominal
    Symmetric Binary
    Assymetric Binary
    Ordinal
    Numeric
    
Compute the dissimilarity matrix for each of the attribute separately
Compute the dissimilarity for the aggregate case
    """
    

import numpy as np
from scipy import sparse,spatial

In [139]:


from random import Random, randint, random, shuffle
from secrets import choice


class Person():
    def __init__(self, name, age = None, sex = None, employed = None, income = None):
        self.name = name # Nominal
        if age:
            self.age = age # Ordinal
        else: self.age = randint(20,90)
        
        
        if not sex: 
            self.sex = choice(seq=['Female','Male']) # Symmetric binary
        else: self.sex = sex
       
        
        if employed == None: 
            self.employed = choice(seq=[True,False]) # Assymetric binary
        else: self.employed = False
            
            
        if self.employed: self.income = randint(0,100000) # Numeric
        else: self.income = 0
        
    def __str__(self):
        if self.employed:
            return f'{self.name} is a {self.sex} at the age of {self.age} and a yearly salary of {self.income}$.'
        else: return f'{self.name} is an unemployed {self.sex} at the age of {self.age}.'
    
    
    
    

         
names = ['John', 'Zoe', 'Rebecka', 'Aleix', 'Gabriel']
sex = ['Male', 'Female', 'Female', 'Male', 'Male']

persons = []
for a,n in enumerate(names):
    persons.append(Person(n,sex =sex[a]))
    print(persons[a])
        

John is a Male at the age of 64 and a yearly salary of 60829$.
Zoe is a Female at the age of 46 and a yearly salary of 89150$.
Rebecka is an unemployed Female at the age of 36.
Aleix is an unemployed Male at the age of 86.
Gabriel is an unemployed Male at the age of 29.


In [140]:
""" Dissimilarity matrix for Name """
mat_name = np.empty((len(persons),len(persons)))

for n in range(len(persons)):
    for k in range(n+1,len(persons)):
        if persons[n].name == persons[k].name:
            mat_name[n][k] = 0
            mat_name[k][n] = 0
        else: 
            mat_name[n][k] = 1
            mat_name[k][n] = 1
print(mat_name)


[[1.49166815e-154 1.00000000e+000 1.00000000e+000 1.00000000e+000
  1.00000000e+000]
 [1.00000000e+000 0.00000000e+000 1.00000000e+000 1.00000000e+000
  1.00000000e+000]
 [1.00000000e+000 1.00000000e+000 0.00000000e+000 1.00000000e+000
  1.00000000e+000]
 [1.00000000e+000 1.00000000e+000 1.00000000e+000 4.94065646e-323
  1.00000000e+000]
 [1.00000000e+000 1.00000000e+000 1.00000000e+000 1.00000000e+000
  0.00000000e+000]]


In [141]:
""" Dissimilarity matrix for sex """
mat_sex = np.empty((len(persons),len(persons)))
for n in range(len(persons)):
    for k in range(n+1,len(persons)):
        if persons[n].name == persons[k].name:
            mat_sex[n][k] = 0
            mat_sex[k][n] = 0
        else: 
            mat_sex[n][k] = 1
            mat_sex[k][n] = 1
print(mat_sex)

[[1.49166815e-154 1.00000000e+000 1.00000000e+000 1.00000000e+000
  1.00000000e+000]
 [1.00000000e+000 0.00000000e+000 1.00000000e+000 1.00000000e+000
  1.00000000e+000]
 [1.00000000e+000 1.00000000e+000 0.00000000e+000 1.00000000e+000
  1.00000000e+000]
 [1.00000000e+000 1.00000000e+000 1.00000000e+000 4.94065646e-323
  1.00000000e+000]
 [1.00000000e+000 1.00000000e+000 1.00000000e+000 1.00000000e+000
  0.00000000e+000]]


In [142]:
""" Dissimilarity matrix for age """

# Rank the persons from youngest to oldest
mat_age = np.empty((len(persons),len(persons)))
persons.sort(key=lambda self: self.age)

for n in range(len(persons)):
    for k in range(n+1,len(persons)):
        z = n/(len(persons)-1)

        mat_age[n][k] = z
        mat_age[k][n] = z

print(mat_age)


[[1.49166815e-154 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 2.50000000e-001 2.50000000e-001
  2.50000000e-001]
 [0.00000000e+000 2.50000000e-001 0.00000000e+000 5.00000000e-001
  5.00000000e-001]
 [0.00000000e+000 2.50000000e-001 5.00000000e-001 4.94065646e-323
  7.50000000e-001]
 [0.00000000e+000 2.50000000e-001 5.00000000e-001 7.50000000e-001
  0.00000000e+000]]


In [143]:
""" Dissimilarity matrix for employment """
mat_employment = np.empty((len(persons),len(persons)))
for n in range(len(persons)):
    for k in range(n+1,len(persons)):
        if persons[n].employed == persons[k].employed:
            mat_employment[n][k] = 0
            mat_employment[k][n] = 0
        else: 
            mat_employment[n][k] = 1
            mat_employment[k][n] = 1
print(mat_employment)


[[1.49166815e-154 0.00000000e+000 1.00000000e+000 1.00000000e+000
  0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 1.00000000e+000 1.00000000e+000
  0.00000000e+000]
 [1.00000000e+000 1.00000000e+000 0.00000000e+000 0.00000000e+000
  1.00000000e+000]
 [1.00000000e+000 1.00000000e+000 0.00000000e+000 4.94065646e-323
  1.00000000e+000]
 [0.00000000e+000 0.00000000e+000 1.00000000e+000 1.00000000e+000
  0.00000000e+000]]


In [144]:
""" Dissimilarity matrix for age """
# Rank the persons from youngest to oldest
persons.sort(key=lambda self: self.age)
mat_income = np.empty((len(persons),len(persons)))
for n in range(len(persons)):
    for k in range(n+1,len(persons)):
        z = spatial.distance.euclidean([persons[n].income], [persons[k].income])

        mat_income[n][k] = z
        mat_income[k][n] = z

print(mat_income)


[[1.49166815e-154 0.00000000e+000 8.91500000e+004 6.08290000e+004
  0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 8.91500000e+004 6.08290000e+004
  0.00000000e+000]
 [8.91500000e+004 8.91500000e+004 0.00000000e+000 2.83210000e+004
  8.91500000e+004]
 [6.08290000e+004 6.08290000e+004 2.83210000e+004 4.94065646e-323
  6.08290000e+004]
 [0.00000000e+000 0.00000000e+000 8.91500000e+004 6.08290000e+004
  0.00000000e+000]]


In [160]:
""" For the agregate case """
matrices = [mat_age,mat_name, mat_employment, mat_income, mat_sex]
cumulative_matrix = np.empty((len(matrices),len(matrices)))
for n in range(len(matrices)):
    for k in range(n,len(matrices)):
        cumulative_matrix[n][k] = (mat_age[n][k] + mat_name[n][k] + mat_employment[n][k] + mat_income[n][k] + mat_sex[n][k])/len(matrices)
    
print(cumulative_matrix)

[[1.49166815e-154 4.00000000e-001 1.78306000e+004 1.21664000e+004
  4.00000000e-001]
 [0.00000000e+000 0.00000000e+000 1.78306500e+004 1.21664500e+004
  4.50000000e-001]
 [8.91500000e+004 8.91500000e+004 0.00000000e+000 5.66470000e+003
  1.78307000e+004]
 [6.08290000e+004 6.08290000e+004 2.83210000e+004 4.94065646e-323
  1.21665500e+004]
 [0.00000000e+000 0.00000000e+000 8.91500000e+004 6.08290000e+004
  0.00000000e+000]]
